In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


lip_readingフォルダへ移動

In [2]:
%cd /content/drive/'My Drive'/lip_reading

/content/drive/My Drive/lip_reading


import

In [3]:
import numpy as np
import glob
import time
import os.path as osp
from PIL import Image
import torch
from torch import nn, optim
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.nn.modules import TransformerEncoder, TransformerEncoderLayer
from torchvision.models import MobileNetV2
from tqdm import tqdm

torch.cuda.is_available()

True

学習するモデルを選ぶ

In [4]:
# LEARN_NETWORK = "Transformer"
LEARN_NETWORK = "LSTM"

モデルの定義

In [5]:
class MobileNetV2NotClassify(MobileNetV2):
    def __init__(self):
        super().__init__()

    def _forward_impl(self, x):
        x = self.features(x)
        # Cannot use "squeeze" as batch-size can be 1 => must use reshape with x.shape[0]
        x = nn.functional.adaptive_avg_pool2d(x, 1).reshape(x.shape[0], -1)
        return x


class Model_Transformer(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.device = device
        self.mobilenet = MobileNetV2NotClassify()
        encoder_layer = TransformerEncoderLayer(1280, 8)
        self.encoder = TransformerEncoder(encoder_layer, 1)
        self.last_dropout = nn.Dropout(0.2)
        self.last_linear = nn.Linear(1280, 4)
        print(self.last_linear.weight)
        nn.init.kaiming_normal_(self.last_linear.weight)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.each_apply(x)
        x = self.encoder(x)
        x = self.last_dropout(x[:, 0, :])
        x = self.last_linear(x)
        output = self.softmax(x)

        return output

    def each_apply(self, x):
        """
            this function means
            result = torch.zeros(x.size()[0], 20, 1280).to(self.device)
            for i in range(0, 20):
                x[:, i] = self.mobilenet(x[:, i])
        """
        result = torch.zeros(x.size()[0], 20, 1280).to(self.device)

        result[:, 0] = self.mobilenet(x[:, 0])
        result[:, 1] = self.mobilenet(x[:, 1])
        result[:, 2] = self.mobilenet(x[:, 2])
        result[:, 3] = self.mobilenet(x[:, 3])
        result[:, 4] = self.mobilenet(x[:, 4])
        result[:, 5] = self.mobilenet(x[:, 5])
        result[:, 6] = self.mobilenet(x[:, 6])
        result[:, 7] = self.mobilenet(x[:, 7])
        result[:, 8] = self.mobilenet(x[:, 8])
        result[:, 9] = self.mobilenet(x[:, 9])
        result[:, 10] = self.mobilenet(x[:, 10])
        result[:, 11] = self.mobilenet(x[:, 11])
        result[:, 12] = self.mobilenet(x[:, 12])
        result[:, 13] = self.mobilenet(x[:, 13])
        result[:, 14] = self.mobilenet(x[:, 14])
        result[:, 15] = self.mobilenet(x[:, 15])
        result[:, 16] = self.mobilenet(x[:, 16])
        result[:, 17] = self.mobilenet(x[:, 17])
        result[:, 18] = self.mobilenet(x[:, 18])
        result[:, 19] = self.mobilenet(x[:, 19])
        return result

class Model_LSTM(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.device = device
        self.mobilenet = MobileNetV2NotClassify()
        self.lstm = nn.LSTM(1280, 1280)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(1280, 4)
        print(self.linear.weight)
        nn.init.kaiming_normal_(self.linear.weight)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.each_apply(x)
        _, (x, _) = self.lstm(x.resize(20, x.size()[0], 1280))
        x = self.dropout(x[0])
        x = self.linear(x)
        output = self.softmax(x)
        return output

    def each_apply(self, x):
        """
            this function means
            result = torch.zeros(x.size()[0], 20, 1280).to(self.device)
            for i in range(0, 20):
                x[:, i] = self.mobilenet(x[:, i])
        """
        result = torch.zeros(x.size()[0], 20, 1280).to(self.device)

        result[:, 0] = self.mobilenet(x[:, 0])
        result[:, 1] = self.mobilenet(x[:, 1])
        result[:, 2] = self.mobilenet(x[:, 2])
        result[:, 3] = self.mobilenet(x[:, 3])
        result[:, 4] = self.mobilenet(x[:, 4])
        result[:, 5] = self.mobilenet(x[:, 5])
        result[:, 6] = self.mobilenet(x[:, 6])
        result[:, 7] = self.mobilenet(x[:, 7])
        result[:, 8] = self.mobilenet(x[:, 8])
        result[:, 9] = self.mobilenet(x[:, 9])
        result[:, 10] = self.mobilenet(x[:, 10])
        result[:, 11] = self.mobilenet(x[:, 11])
        result[:, 12] = self.mobilenet(x[:, 12])
        result[:, 13] = self.mobilenet(x[:, 13])
        result[:, 14] = self.mobilenet(x[:, 14])
        result[:, 15] = self.mobilenet(x[:, 15])
        result[:, 16] = self.mobilenet(x[:, 16])
        result[:, 17] = self.mobilenet(x[:, 17])
        result[:, 18] = self.mobilenet(x[:, 18])
        result[:, 19] = self.mobilenet(x[:, 19])
        return result

データローダー、Utility Class, Functionの定義

In [6]:
class ImageTransform():
    def __init__(self, size, mean, std):
        self.data_transform = transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ])
    def __call__(self, data):
        result = torch.Tensor(20, 3, 224, 224)
        for i in range(0, 20):
            if data[i, 0, 0, 0] == -1:
                prev_img = self.data_transform(Image.fromarray(data[i - 1].astype(np.uint8)))
                for j in range(i, 20):
                    result[j] = prev_img
                return result
            else:
                result[i] = self.data_transform(Image.fromarray(data[i].astype(np.uint8)))
        return result

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = np.load(img_path)

        img_transformed = self.transform(img)

        label = int(img_path[10]) - 1

        return img_transformed, label


def make_datapath_list():
    rootpath = "./dataset/"
    path_list = []

    for i in range(1,5):
        target_path = osp.join(rootpath + str(i) + '/*.npy')
        for path in glob.glob(target_path):
            path_list.append(path)

    return train_test_split(np.array(path_list), train_size=0.8)

def train_model(net, device, dataloaders_dict, criterion, optimizer, num_epochs):
  start = None
  end = None
  net.to(device)

  torch.backends.cudnn.benchmark = True

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-----------------------------')

    for phase in ['train', 'test']:
      if phase == 'train':
        start = time.time()
        net.train()
      else:
        net.eval()

      epoch_loss = 0.0
      epoch_corrects = 0.0

      for inputs, labels in tqdm(dataloaders_dict[phase]):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
          outputs = net(inputs)
          loss = criterion(outputs, labels)
          _, preds = torch.max(outputs, dim=1)

          if phase == 'train':
            loss.backward()
            optimizer.step()

          epoch_loss += loss.item() * inputs.size(0)

          epoch_corrects += torch.sum(preds == labels.data)

      if phase == 'train':
        end = time.time()
        print("Epock {}| {}s".format(epoch+1, end - start))
      epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
      epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
  
  if LEARN_NETWORK == "Transformer":
    save_path = './transformer_weights.pth'
  else:
    save_path = './lstm_weights.pth'
  torch.save(net.state_dict(), save_path)


実行

In [ ]:
size = (224, 224)
mean = (0.485, 0.486, 0.406)
std = (0.229, 0.224, 0.225)

train_path, test_path = make_datapath_list()

batch_size = 8

transform = ImageTransform(size, mean, std)

train_dataset = MyDataset(train_path, transform)
test_dataset = MyDataset(test_path, transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_dataloader, "test": test_dataloader}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス", device)

if LEARN_NETWORK == "Transformer":
  net = Model_Transformer(device) # Transformerを使っているモデル
else:
  net = Model_LSTM(device) # LSTMを使っているモデル
optimizer = optim.SGD(net.parameters(), lr=1e-2, momentum=0.9)
criterion = nn.CrossEntropyLoss()
num_epochs = 20
train_model(net, device, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス cuda:0
Parameter containing:
tensor([[-0.0190,  0.0151, -0.0087,  ...,  0.0167, -0.0255,  0.0274],
        [-0.0187, -0.0188,  0.0074,  ..., -0.0173, -0.0131,  0.0019],
        [ 0.0066,  0.0048, -0.0267,  ...,  0.0136, -0.0055,  0.0110],
        [-0.0114,  0.0138,  0.0143,  ...,  0.0108,  0.0160, -0.0089]],
       requires_grad=True)


  0%|          | 0/202 [00:00<?, ?it/s]

Epoch 1/20
-----------------------------


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:358: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
  0%|          | 0/51 [00:00<?, ?it/s]

Epock 1| 734.9281709194183s
train Loss: 1.3965 Acc: 0.2673


...

  0%|          | 0/51 [00:00<?, ?it/s]

Epock 5| 420.90414094924927s
train Loss: 1.3928 Acc: 0.2580


  0%|          | 0/202 [00:00<?, ?it/s]

test Loss: 1.3875 Acc: 0.2624
Epoch 6/20
-----------------------------


 45%|████▌     | 91/202 [02:42<03:00,  1.63s/it]